In [1]:
import json
from datetime import date
import polars as pl
import numpy as np

In [2]:
schema={
    'id': pl.Utf8,
    'abstract': pl.Utf8,
    'beratungsstand': pl.Utf8,
    'vorgangstyp': pl.Utf8,
    'gesta': pl.Utf8,
    'archiv': pl.Utf8,
    'sachgebiet': pl.List(pl.Utf8),
    'typ': pl.Utf8,
    'wahlperiode': pl.UInt64,
    'zustimmungsbeduerftigkeit': pl.List(pl.Utf8),
    'initiative': pl.List(pl.Utf8),
    'deskriptor': pl.List(pl.Struct([
        pl.Field('fundstelle', pl.Boolean),
        pl.Field('name', pl.Utf8),
        pl.Field('typ', pl.Utf8),
    ])),
    'aktualisiert': pl.Utf8,
    'titel': pl.Utf8,
    'datum': pl.Utf8,
    'verkuendung': pl.List(pl.Struct([
        pl.Field('verkuendungsdatum', pl.Utf8)
    ]))
}
df_vorgang = pl.DataFrame(json.load(open('vorgang.json', encoding='utf-8')), schema=schema)
df_gesetze = df_vorgang.filter(pl.col('vorgangstyp') == 'Gesetzgebung')
pl.Config(fmt_str_lengths=100)
df_gesetze['beratungsstand'].unique()

beratungsstand
str
"""1. Durchgang im Bundesrat abgeschlossen"""
"""Für erledigt erklärt"""
"""Bundesrat hat zugestimmt"""
"""Bundesrat hat Zustimmung versagt"""
"""Verabschiedet"""
"""Im Vermittlungsverfahren"""
"""Bundesrat hat keinen Einspruch eingelegt"""
"""Beschlussempfehlung liegt vor"""
"""Einbringung abgelehnt"""


In [3]:
df_vorgangspos = pl.read_json('vorgangsposition.json')
df_vorgangspos = df_vorgangspos.with_columns(
   pl.col("datum").str.to_date(format="%Y-%m-%d")
)
df_vorgangspos = df_vorgangspos.filter((pl.col('vorgangstyp') == 'Gesetzgebung') & (pl.col('typ') == 'Vorgangsposition'))
df_vorgang_duration = df_vorgangspos.group_by(['vorgang_id']).agg([
    pl.col('datum').min().alias('vorgang_start'), 
])

In [4]:
df_gesetze.head()

id,abstract,beratungsstand,vorgangstyp,gesta,archiv,sachgebiet,typ,wahlperiode,zustimmungsbeduerftigkeit,initiative,deskriptor,aktualisiert,titel,datum,verkuendung
str,str,str,str,str,str,list[str],str,u64,list[str],list[str],list[struct[3]],str,str,str,list[struct[1]]
"""308776""","""Zustimmung zu dem am 21. Juli 2023 in Lauterbourg unterzeichneten Abkommen: Standardisierung grenzü…","""Dem Bundestag zugeleitet - Noch nicht beraten""","""Gesetzgebung""","""XK002""",null,"[""Arbeit und Beschäftigung"", ""Bildung und Erziehung""]","""Vorgang""",20,"[""Ja, laut Gesetzentwurf (Drs 76/24)""]","[""Bundesregierung""]","[{false,""Ausbildungsvertrag"",""Sachbegriffe""}, {false,""Auswärtige Kultur- und Bildungspolitik"",""Sachbegriffe""}, … {false,""Teilzeitarbeit"",""Sachbegriffe""}]","""2024-03-25T15:18:40+01:00""","""Gesetz zu dem Abkommen vom 21. Juli 2023 zwischen der Regierung der Bundesrepublik Deutschland und …","""2024-03-25""",null
"""308709""","""Zusammenfassung und anwendungsbezogene Konkretisierung der sich aus der Anwendung des BEPS-MLI erge…","""Dem Bundestag zugeleitet - Noch nicht beraten""","""Gesetzgebung""","""D057""",null,"[""Öffentliche Finanzen, Steuern und Abgaben""]","""Vorgang""",20,"[""Ja, laut Gesetzentwurf (Drs 75/24)""]","[""Bundesregierung""]","[{true,""BEPS-MLI-Anwendungsgesetz"",""Rechtsmaterialien""}, {false,""Bundeszentralamt für Steuern"",""Institutionen""}, … {false,""Unternehmensbesteuerung"",""Sachbegriffe""}]","""2024-03-25T15:24:10+01:00""","""Gesetz zur Anwendung des Mehrseitigen Übereinkommens vom 24. November 2016 und zu weiteren Maßnahme…","""2024-03-25""",null
"""308703""","""Zur Sicherstellung einer tat- und schuldangemessenen Reaktion im Einzelfall bei Fallkonstellationen…","""Überwiesen""","""Gesetzgebung""","""C084""",null,"[""Gesellschaftspolitik, soziale Gruppen"", ""Recht""]","""Vorgang""",20,"[""Nein, laut Gesetzentwurf (Drs 74/24 (bes.eilbed.))""]","[""Bundesregierung""]","[{true,""Gesetz zur Anpassung der Mindeststrafen des § 184b Absatz 1 Satz 1 und Absatz 3 des Strafgesetzbuches"",""Rechtsmaterialien""}, {false,""Gesetz zur Bekämpfung sexualisierter Gewalt gegen Kinder"",""Rechtsmaterialien""}, … {false,""Strafzumessung"",""Sachbegriffe""}]","""2024-03-25T13:28:46+01:00""","""Gesetz zur Anpassung der Mindeststrafen des § 184b Absatz 1 Satz 1 und Absatz 3 des Strafgesetzbuch…","""2024-03-25""",null
"""308700""","""Umsetzung von EU-Regelungen im Bereich der GAP betr. Einführung der sozialen Konditionalität zur En…","""Dem Bundestag zugeleitet - Noch nicht beraten""","""Gesetzgebung""","""F017""",null,"[""Landwirtschaft und Ernährung""]","""Vorgang""",20,"[""Nein, laut Gesetzentwurf (Drs 70/24)""]","[""Bundesregierung""]","[{true,""Agrarsubvention"",""Sachbegriffe""}, {false,""Arbeits- und Sozialstandards"",""Sachbegriffe""}, … {false,""Verordnung der EU"",""Sachbegriffe""}]","""2024-03-25T15:21:09+01:00""","""Erstes Gesetz zur Änderung des GAP-Konditionalitäten-Gesetzes""","""2024-03-25""",null
"""307621""","""Behebung struktureller Probleme im Bundeshaushalt und Einleitung der qualitativen Konsolidierung zu…","""Bundesrat hat Vermittlungsausschuss nicht angerufen""","""Gesetzgebung""","""D056""",null,"[""Öffentliche Finanzen, Steuern und Abgaben""]","""Vorgang""",20,"[""Nein, laut Gesetzentwurf (Drs 20/9999)""]","[""Fraktion BÜNDNIS 90/DIE GRÜNEN"", ""Fraktion der FDP"", ""Fraktion der SPD""]","[{false,""Agrardiesel"",""Sachbegriffe""}, {false,""Arbeitslosenversicherung"",""Sachbegriffe""}, … {false,""Öffentliche Einnahmen"",""Sachbegriffe""}]","""2024-03-22T11:56:31+01:00""","""Zweites Haushaltsfinanzierungsgesetz 2024""","""2024-03-22""",null


In [5]:
df_gesetze = df_gesetze.join(df_vorgang_duration.select(['vorgang_id', 'vorgang_start']), left_on='id', right_on='vorgang_id', how='left')

In [6]:
today = date.today()
df_gesetze = df_gesetze.with_columns(
    pl.when(pl.col('beratungsstand') == 'Verkündet').then(
        pl.col('verkuendung').list.eval(pl.col('').struct.field('verkuendungsdatum').str.to_date(format="%Y-%m-%d")).list.min()
    ).when(pl.col('beratungsstand') == 'Abgelehnt').then(
        pl.col('datum').str.to_date(format="%Y-%m-%d")
    ).otherwise(today).alias('vorgang_end')
)
df_gesetze = df_gesetze.with_columns((pl.col('vorgang_end') - pl.col('vorgang_start')).dt.total_days().alias('vorgangsdauer'))

In [7]:
reg_frag = ['Fraktion BÜNDNIS 90/DIE GRÜNEN', 'Fraktion der FDP', 'Fraktion der SPD']

def fix_initiative(x):
    return pl.when(x.list.set_intersection(reg_frag).len == len(reg_frag)).then(
        x.list.set_difference(reg_frag)
    ).otherwise(x)

df_gesetze = df_gesetze.with_columns(pl.when(pl.col('initiative').list.set_intersection(reg_frag).list.len() == len(reg_frag)).then(
        pl.col('initiative').list.set_difference(reg_frag).list.concat(pl.Series(['Bundesregierung']))
    ).otherwise(pl.col('initiative')).alias('initiative'))

In [8]:
df_gesetze = df_gesetze.with_columns(pl.col('deskriptor').list.eval(pl.element().struct.field('name')).alias('keywords'))

In [9]:
df_gesetze.select(pl.col("*").exclude("abstract", "vorgangstyp", "gesta", "archiv", "typ", "deskriptor")).write_json('gesetze.json', row_oriented=True)

In [10]:
df_gesetze_public = df_gesetze.filter(pl.col('beratungsstand') == 'Verkündet')
bins = [7, 14, 30, 60, 120, 240, 365]
duration_bin = np.digitize(df_gesetze_public.select('vorgangsdauer').to_series(), bins, right=True)
df_gesetze_public.select('vorgangsdauer').to_series().hist(bins)

break_point,category,vorgangsdauer_count
f64,cat,u32
7.0,"""(-inf, 7.0]""",1
14.0,"""(7.0, 14.0]""",3
30.0,"""(14.0, 30.0]""",19
60.0,"""(30.0, 60.0]""",22
120.0,"""(60.0, 120.0]""",70
240.0,"""(120.0, 240.0]""",103
365.0,"""(240.0, 365.0]""",5
inf,"""(365.0, inf]""",1


In [11]:
df_gesetze_public_bins = df_gesetze_public.with_columns(pl.from_numpy(duration_bin, schema=['duration_bin']))
df_gesetze_public_bins.filter(pl.col('duration_bin') == pl.col('duration_bin').min())

id,abstract,beratungsstand,vorgangstyp,gesta,archiv,sachgebiet,typ,wahlperiode,zustimmungsbeduerftigkeit,initiative,deskriptor,aktualisiert,titel,datum,verkuendung,vorgang_start,vorgang_end,vorgangsdauer,keywords,duration_bin
str,str,str,str,str,str,list[str],str,u64,list[str],list[str],list[struct[3]],str,str,str,list[struct[1]],date,date,i64,list[str],i64
"""283063""","""Impfpflicht gegen COVID-19 bzw. Nachweis über Genesung oder Kontraindikation für Beschäftigte besti…","""Verkündet""","""Gesetzgebung""","""M002""","""XX/2""","[""Gesundheit""]","""Vorgang""",20,"[""Ja, laut Gesetzentwurf (Drs 20/188)"", ""Ja, laut Verkündung (BGBl I)""]","[""Bundesregierung""]","[{false,""Apotheker"",""Sachbegriffe""}, {false,""Arbeitnehmer-Entsendegesetz"",""Rechtsmaterialien""}, … {false,""Zahnarzt"",""Sachbegriffe""}]","""2022-07-26T19:57:25+02:00""","""Gesetz zur Stärkung der Impfprävention gegen COVID-19 und zur Änderung weiterer Vorschriften im Zus…","""2021-12-10""","[{""2021-12-11""}]",2021-12-06,2021-12-11,5,"[""Apotheker"", ""Arbeitnehmer-Entsendegesetz"", … ""Zahnarzt""]",0


In [12]:
df_gesetze_public_bins.filter(pl.col('duration_bin') == pl.col('duration_bin').max())

id,abstract,beratungsstand,vorgangstyp,gesta,archiv,sachgebiet,typ,wahlperiode,zustimmungsbeduerftigkeit,initiative,deskriptor,aktualisiert,titel,datum,verkuendung,vorgang_start,vorgang_end,vorgangsdauer,keywords,duration_bin
str,str,str,str,str,str,list[str],str,u64,list[str],list[str],list[struct[3]],str,str,str,list[struct[1]],date,date,i64,list[str],i64
"""297317""",null,"""Verkündet""","""Gesetzgebung""","""XB002""","""XX/225""","[""Europapolitik und Europäische Union""]","""Vorgang""",20,"[""Ja, laut Gesetzentwurf (Drs 104/23)"", ""Ja, laut Verkündung (BGBl II)""]","[""Bundesregierung""]","[{false,""Europawahl"",""Sachbegriffe""}, {false,""Europäisches Parlament"",""Institutionen""}, {false,""Sperrklausel"",""Sachbegriffe""}]","""2024-03-13T16:36:41+01:00""","""Gesetz zu dem Beschluss (EU, Euratom) 2018/994 des Rates der Europäischen Union vom 13. Juli 2018 z…","""2023-07-07""","[{""2024-03-13""}]",2023-03-10,2024-03-13,369,"[""Europawahl"", ""Europäisches Parlament"", ""Sperrklausel""]",7
